<a href="https://colab.research.google.com/github/Gandulf78/generate_podcast/blob/main/podcast_creator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Utilitaire de génération de Podcast

Cliquer sur "Tout exécuter" puis quand les boutons s'affichent sélectionner votre fichier de script avec Upload() puis Générer le podcast.

In [ ]:
# @title
!pip install -q git+https://github.com/suno-ai/bark.git
!pip install -q soundfile ipywidgets

import os
import torch
import ipywidgets as widgets
from IPython.display import display, clear_output

import warnings
import logging
# Avant toute importation de Bark
import tqdm
from tqdm.notebook import tqdm as notebook_tqdm

# Remplacer toutes les références à tqdm par la version notebook
tqdm.tqdm = notebook_tqdm
tqdm.std.tqdm = notebook_tqdm

from bark.generation import preload_models

# 1. Configurer le logging global pour tout capturer dans un fichier
logging.basicConfig(
    filename='bark_generation.log',
    filemode='w',  # ou 'a' si tu veux ajouter
    level=logging.WARNING,
    format='[%(asctime)s] %(levelname)s:%(name)s: %(message)s'
)

# 2. Supprimer tous les handlers du logger racine (souvent configuré par défaut)
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# 3. Ajouter un FileHandler propre
file_handler = logging.FileHandler('bark_generation.log', mode='w')
file_handler.setLevel(logging.WARNING)
formatter = logging.Formatter('[%(asctime)s] %(levelname)s:%(name)s: %(message)s')
file_handler.setFormatter(formatter)
logging.root.addHandler(file_handler)

# 4. Rediriger les warnings classiques vers logging
def custom_warning_to_log(message, category, filename, lineno, file=None, line=None):
    logging.warning(f"{category.__name__} in {filename}:{lineno} — {message}")

warnings.showwarning = custom_warning_to_log

# Détection device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"🎯 Device utilisé : {device}")

# Environnement Bark
os.environ["SUNO_USE_SMALL_MODELS"] = "True"
os.environ["TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD"] = "1"

# Lire la variable d'environnement (secret)
from google.colab import userdata
try:
  use_drive = userdata.get('USE_DRIVE')
except KeyError:
  use_drive = False

output_dir = "/content"

def setup_output_dir(change=None):
    global output_dir
    if use_drive.value:
        from google.colab import drive
        drive.mount('/content/drive')
        output_dir = "/content/drive/MyDrive/BarkOutputs"
    else:
        output_dir = "/content"
    os.makedirs(output_dir, exist_ok=True)
    print(f"📁 Dossier de sortie : {output_dir}")

# Chargement des modèles
preload_models()
print("✅ Modèles chargés")


In [ ]:
# @title
import numpy as np
import re
import soundfile as sf
from bark.generation import generate_text_semantic
from bark.api import semantic_to_waveform

SAMPLE_RATE = 24000
GEN_TEMP = 0.6
silence = np.zeros(int(0.25 * SAMPLE_RATE), dtype=np.float32)

# Voix
voice_prompts = {
    "John": "v2/en_speaker_1",
    "Samantha": "v2/en_speaker_9" # Changed to a different female voice prompt
}

# Fonctions
def parse_script(file_path):
    pattern = re.compile(r"(John|Samantha):\s*(.*)")
    parsed = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            m = pattern.match(line.strip())
            if m:
                parsed.append((m.group(1), m.group(2)))
    return parsed

def generate_audio(parsed_lines):
    pieces = []

    # Affichage d'une barre de progression lisible
    for speaker, text in parsed_lines:
        sem = generate_text_semantic(
            text,
            history_prompt=voice_prompts[speaker],
            temp=GEN_TEMP,
            min_eos_p=0.05
        )
        audio = semantic_to_waveform(sem, history_prompt=voice_prompts[speaker])
        pieces.append(audio)
        pieces.append(silence.copy())

    return np.concatenate(pieces)

# Interface
upload = widgets.FileUpload(accept=".txt", multiple=False)
generate_button = widgets.Button(description="🎙️ Générer le podcast")
output_widget = widgets.Output()

def on_generate_clicked(b):
    with output_widget:
        clear_output()
        if not upload.value:
            print("⚠️ Aucun fichier script.txt téléversé.")
            return

        for name, file in upload.value.items():
            path = f"/content/{name}"
            with open(path, "wb") as f:
                f.write(file["content"])
            parsed = parse_script(path)
            print("🗣️ Contenu du script :")
            for spk, txt in parsed:
                print(f"{spk}: {txt}")
            audio_array = generate_audio(parsed)
            wav_path = os.path.join(output_dir, "podcast_output.wav")
            sf.write(wav_path, audio_array, SAMPLE_RATE)
            print(f"✅ Audio généré : {wav_path}")

            if "drive" not in wav_path:
                from google.colab import files
                files.download(wav_path)

generate_button.on_click(on_generate_clicked)

display(upload, generate_button, output_widget)